In [3]:
import torch
import torch.nn as nn
# from apex import amp
from torch.cuda import amp
import numpy as np
from tqdm import tqdm
import torch
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transforms
from torch.autograd import Variable
from PIL import Image
from matplotlib.pyplot import imshow
from scipy import ndimage as ndi
from skimage.transform import resize
import pandas as pd
import os
import nibabel as nib


import matplotlib.pyplot as plt
%matplotlib inline

%gui qt

In [4]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [5]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!nvidia-smi

Mon May 17 20:47:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 430.50       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:00:05.0 Off |                  N/A |
|  0%   20C    P0    53W / 250W |      0MiB / 11178MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device= torch.device('cpu')
device

device(type='cuda')

# Import Data

In [7]:
dataDir = 'downloads/OpenDataset/'
# dataDir = 'drive/My Drive/B.Sc proj/OpenDataset/'
metadata = pd.read_csv(dataDir + '201014_M&Ms_Dataset_Information_-_opendataset.csv')
metadata

,External code,VendorName,Vendor,Centre,ED,ES
0,A0S9V9,Siemens,A,1,0,9
1,A1D0Q7,Philips,B,2,0,9
2,A1D9Z7,Siemens,A,1,22,11
3,A1E9Q1,Siemens,A,1,0,9
4,A1K2P5,Canon,D,5,33,11
...,...,...,...,...,...,...
340,T2Z1Z9,Canon,D,5,29,9
341,T9U9W2,Siemens,A,1,0,10
342,V4W8Z5,GE,C,4,19,9
343,W5Z4Z8,Philips,B,2,29,11


In [12]:
def save_csv(all_dirs, file_name, addr):    
    for idx in range(len(all_dirs)):
        if idx%2 == 0:
            res = pd.DataFrame(columns=['img', 'gt', 'id', 'ES-ED'])
        
        item = all_dirs[idx]
        path = addr + item

        mask, img, id = None, None, None

        for root, dir, files in os.walk(path):
            for f in files:
                img_arr = nib.load(path + '/' + f).get_fdata()
#                 print(f"file name is {f}")
                if f[-8] == 't':
                    id = f.split('_')[0]
                    mask = img_arr
                else:
                    img = img_arr
        
        # preparing data 
#         print(f"df: {metadata[metadata['External code'] == id]['ES']}")
        print(f'{idx} out of {len(all_dirs)}')
        t = metadata[metadata['External code'] == id]['ES'].iloc[0]
        img1 = img[:, :, :, t]
        gt1 = mask[:, :, :, t]  
        my_dict = {'img': img1, 'gt': gt1, 'id': id, 'ES-ED': 'ES'}
        res = res.append(my_dict, ignore_index=True)
        
        t = metadata[metadata['External code'] == id]['ED'].iloc[0]
        img2 = img[:, :, :, t]
        gt2 = mask[:, :, :, t]
        my_dict = {'img': img2, 'gt': gt2, 'id': id, 'ES-ED': 'ED'}
        res = res.append(my_dict, ignore_index=True)
        
        if idx % 2 == 1:
            res.to_csv(f'{file_name}_batch_{idx // 2}.csv' , index=False)    
    
    res.to_csv(f'{file_name}_batch_{idx // 2}.csv' , index=False)

In [16]:
train_dir = dataDir + '/Training/Labeled/'
train_all = os.listdir(train_dir)
save_csv(train_all, 'train/train', train_dir)

0 out of 150
1 out of 150
2 out of 150
3 out of 150
4 out of 150
5 out of 150
6 out of 150
7 out of 150
8 out of 150
9 out of 150
10 out of 150
11 out of 150
12 out of 150
13 out of 150
14 out of 150
15 out of 150
16 out of 150
17 out of 150
18 out of 150
19 out of 150
20 out of 150
21 out of 150
22 out of 150
23 out of 150
24 out of 150
25 out of 150
26 out of 150
27 out of 150
28 out of 150
29 out of 150
30 out of 150
31 out of 150
32 out of 150
33 out of 150
34 out of 150
35 out of 150
36 out of 150
37 out of 150
38 out of 150
39 out of 150
40 out of 150
41 out of 150
42 out of 150
43 out of 150
44 out of 150
45 out of 150
46 out of 150
47 out of 150
48 out of 150
49 out of 150
50 out of 150
51 out of 150
52 out of 150
53 out of 150
54 out of 150
55 out of 150
56 out of 150
57 out of 150
58 out of 150
59 out of 150
60 out of 150
61 out of 150
62 out of 150
63 out of 150
64 out of 150
65 out of 150
66 out of 150
67 out of 150
68 out of 150
69 out of 150
70 out of 150
71 out of 150
72

In [18]:
val_dir = dataDir + '/Validation/'
val_all = os.listdir(val_dir)
save_csv(val_all, 'val/val', val_dir)

0 out of 34
1 out of 34


FileNotFoundError: [Errno 2] No such file or directory: 'val/val_batch_0.csv'